<a href="https://colab.research.google.com/github/mimi-1/DataProjects/blob/main/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The dataset was downloaded from Kaggle  
[Cardiovascular disease data ](https://www.kaggle.com/datasets/sulianova/cardiovascular-disease-dataset)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pylab as plt
import numpy as np #for infinity

!pip install dmba
from dmba import classificationSummary, gainsChart

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 36.1 MB/s eta 0:00:00
no display found. Using non-interactive Agg backend


In [ ]:
plt.style.use('classic')
%matplotlib inline

In [2]:
# file downloaded from Kaggle at Cardiovascular disease data
try:
  cardio_df = pd.read_csv("https://raw.githubusercontent.com/mimi-1/Datasets/main/cardio_train.csv",sep=";",index_col=0)
except Exception as err:
    print(f"Unexpected {err=}, {type(err)=}")
else:
    print("Data were loaded to cardio_df")
    backup = cardio_df.copy(deep=True)

Data were loaded to cardio_df


In [5]:
#transform dataset 

#Remove outliers 

fields_with_outliers = ["ap_lo","ap_hi", "weight", "height"]

for field in fields_with_outliers:
  q_l = int(cardio_df[field].quantile(q=0.005))
  q_h = int(cardio_df[field].quantile(q=0.995))
  cardio_df = cardio_df[ (cardio_df[field] > q_l) & (cardio_df[field] < q_h) ]
# cardio_df.describe()


In [6]:
#integer to categorical 
# create a dictionary to map integer values to categorical values
gender_map={1:'female',2:'male'}
cholesterol_map = {1: 'normal', 2: 'above normal', 3: 'well above normal'}
glucose_map={1: 'normal', 2: 'above average', 3: 'well above normal'}

cardio_df['gender'] = cardio_df['gender'].replace(gender_map)
cardio_df['cholesterol'] = cardio_df['cholesterol'].replace(cholesterol_map)
cardio_df['gluc'] = cardio_df['gluc'].replace(glucose_map)

#Changing object datataypes to categorical
cardio_df['cholesterol'] = cardio_df['cholesterol'].astype('category')
cardio_df['gluc'] = cardio_df['gluc'].astype('category')
cardio_df['gender'] = cardio_df['gender'].astype('category')

In [9]:
#binary to categorical

#Mapping binary 
binary_map = {0: 'no', 1: 'yes'}
binary_fields = ['alco','smoke','active', 'cardio']
for afield in binary_fields:
  cardio_df[afield] = cardio_df[afield].replace(binary_map)
  cardio_df[afield] = cardio_df[afield].astype('category')


In [11]:
cardio_df.head(9)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,male,168,62.0,110,80,normal,normal,no,no,yes,no
1,20228,female,156,85.0,140,90,well above normal,normal,no,no,yes,yes
3,17623,male,169,82.0,150,100,normal,normal,no,no,yes,yes
8,21914,female,151,67.0,120,80,above normal,above average,no,no,no,no
9,22113,female,157,93.0,130,80,well above normal,normal,no,no,yes,no
12,22584,male,178,95.0,130,90,well above normal,well above normal,no,no,yes,yes
15,22530,female,169,80.0,120,80,normal,normal,no,no,yes,no
16,18815,male,173,60.0,120,80,normal,normal,no,no,yes,no
18,14791,male,165,60.0,120,80,normal,normal,no,no,no,no


In [14]:
backup = cardio_df.copy(deep=True)

In [15]:
cardio_df.describe()

,age,height,weight,ap_hi,ap_lo,BMI
count,50704.000000,50704.000000,50704.000000,50704.000000,50704.000000,50704.000000
mean,53.790470,164.789188,74.752010,129.000986,83.972882,27.578152
std,6.597064,7.124585,12.663361,12.762610,6.119067,4.730058
min,30.000000,149.000000,50.000000,101.000000,71.000000,15.090000
25%,50.000000,160.000000,65.000000,120.000000,80.000000,24.030000
50%,54.000000,165.000000,73.000000,120.000000,80.000000,26.610000
75%,59.000000,170.000000,82.000000,140.000000,90.000000,30.410000
max,65.000000,183.000000,116.000000,169.000000,109.000000,49.560000


In [19]:
#Create BMI
#Create variable containing bodymass index BMI from 15 to 50 , we will create 
cardio_df["BMI"] = round(cardio_df["weight"]/((0.01*cardio_df["height"])**2),2)
#Create bins for BMI

bins = [-np.inf,18.5, 25, 30, np.inf]
labels = ["underweight","healthy","overweight","obese"]
cardio_df.BMI = pd.cut(cardio_df.BMI , bins=bins, labels=labels)
cardio_df.BMI = cardio_df.BMI.astype('category')

In [20]:
cardio_df.head(4)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,BMI
id,,,,,,,,,,,,,
0,50.0,male,168,62.0,110,80,normal,normal,no,no,yes,no,healthy
1,55.0,female,156,85.0,140,90,well above normal,normal,no,no,yes,yes,obese
3,48.0,male,169,82.0,150,100,normal,normal,no,no,yes,yes,overweight
8,60.0,female,151,67.0,120,80,above normal,above average,no,no,no,no,overweight


In [ ]:
#transfor age to years
cardio_df["age"] = round(cardio_df["age"]/365,0)

In [21]:
#Create bins for age so bining is a must (age from 30 to 65) 35 groups now

cardio_df['age'] = [round(t / 35) for t in cardio_df['age']] # groups: below 52(including) and above 52
cardio_df['age'] = cardio_df['age'].astype('category')

cardio_df.head(5)



,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,BMI
id,,,,,,,,,,,,,
0,1,male,168,62.0,110,80,normal,normal,no,no,yes,no,healthy
1,2,female,156,85.0,140,90,well above normal,normal,no,no,yes,yes,obese
3,1,male,169,82.0,150,100,normal,normal,no,no,yes,yes,overweight
8,2,female,151,67.0,120,80,above normal,above average,no,no,no,no,overweight
9,2,female,157,93.0,130,80,well above normal,normal,no,no,yes,no,obese


### NAIVE BAYES classification

In [ ]:
#Split the data into training (60%) and testing (40%) 

predictors = ['DAY_WEEK', 'CRS_DEP_TIME', 'ORIGIN', 'DEST', 'CARRIER']
outcome = 'Flight Status'

X = pd.get_dummies(delays_df[predictors])
y = delays_df['Flight Status']
classes = ['ontime', 'delayed']

# split into training and validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)

In [ ]:
# run naive Bayes
delays_nb = MultinomialNB(alpha=0.01)
delays_nb.fit(X_train, y_train)

# predict probabilities (Shows the belonging probabilities of each record to which class)
predProb_train = delays_nb.predict_proba(X_train)
predProb_valid = delays_nb.predict_proba(X_valid)

# predict class membership (shows the class instead of probability by selecting the class with highest probability)
y_valid_pred = delays_nb.predict(X_valid)
y_train_pred = delays_nb.predict(X_train)


In [ ]:
# Use the model to predict a new data

df = pd.concat([pd.DataFrame({'actual': y_valid, 'predicted': y_valid_pred})], axis=1)

mask = ((X_valid.CARRIER_DL == 1) & (X_valid.DAY_WEEK_7 == 1) & (X_valid.CRS_DEP_TIME_10 == 1) & 
        (X_valid.DEST_LGA == 1) & (X_valid.ORIGIN_DCA == 1))

print(df[mask])